In [2]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd

I download the Wiki page and rea it with BeautifoulSoup

In [3]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(URL)
soup = BeautifulSoup(response.text,'html.parser')

In [4]:
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [5]:
rows = table.find_all('tr')

In [6]:
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]

In [7]:
print(columns)

['Postcode', 'Borough', 'Neighbourhood']


I create the dataframe with the postal codes.
The last line of the cell is the onluy way I found to skip di "\n" from the Neighbourhood

In [8]:
df = pd.DataFrame(columns=columns)

In [23]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 4:
        values = [tds[0].text, tds[1].text,'', tds[2].text.replace('\n',''), tds[3].replace('\n','')]
    else:
        values = [td.text.replace('\n','') for td in tds]
  

    df = df.append(pd.Series(values, index = columns), ignore_index=True)
    df['Neighbourhood'].replace('\n','')
    

In [24]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


I drop the lines where the Borough value is "Not assigned"

In [26]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [28]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


I group all the neighbourhoods with the same Borough

In [50]:
df1 = df.groupby("Postcode").agg(lambda x:','.join(set(x)))

In [45]:
df1.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
M1E,Scarborough,"Guildwood,West Hill,Morningside"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


Finally I assign the value in the Borough cell to Neighbouhoods with "Not assigned" value

In [51]:
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']

In [46]:
df1.head(10)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
M1E,Scarborough,"Guildwood,West Hill,Morningside"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Ionview,East Birchmount Park,Kennedy Park"
M1L,Scarborough,"Oakridge,Golden Mile,Clairlea"
M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West"


I check if in the 9th cell in the table the value of the Borough and of the Neighborhood columns are both "Queen's Park"

In [52]:
df1.loc[df1['Borough'] == "Queen's Park"]

,Borough,Neighbourhood
Postcode,,
M7A,Queen's Park,Queen's Park


Finally, I check the shape of the dataframe

In [53]:
df1.shape

(103, 2)